In [1]:
import numpy as np
import math

import random
filepath = 'C:\\Users\\86136\\Desktop\\杂物\\人工智能与机器学习\\作业\\作业7\\lenses.data'



class DecisionTree:
    #def __init__(self, n_feature = 1, n_iter = 200, lr = 1e-3, tol = None):
        #self.n_feature = n_feature
    



    cycle = 0 #记录目前递归次数

    i = 0#帮助控制分支长度
    processed_model = []
    t_type = []


    def predict(self,x):    #输入数组x（n*4),预测n个y值作为列表输出
        y = []
        #print('self.processed_model:',self.processed_model)
        for i in x:
            y.append(self.predict_one_value(i))
        return y


    def predict_one_value(self,x):#单个样本预测
        y = None
        #print(f"self.processed_model",self.processed_model)
        for i in self.processed_model:
            h = 0
            while h < len(i[2]):
                j = i[2][h]    
                h = h+1           
                #print(f"\n\ni:{i}\nx:{x}\nj:{j}\nx[j]:{x[j]}")
                if i[0][j] != x[j]:
                    #print(f"\n\ni:{i}\nx:{x}\nj:{j}\nx[j]:{x[j]}\ni[0][j]:{i[0][j]}")
                    break
                else:
                    y = i[1]
                    if h == len(i[2]):
                        return i[1]#成功匹配则输出，#同一个x对应多个target，取第一个target作为预测值   
        olnm = random.randrange(len(self.t_type))
        #print("self.t_type:",self.t_type)
        if y != None:#(部分特征值匹配)
            return y
        return list(self.t_type)[olnm]#无匹配值，随机输出一个预测值
    
    def model_process(self):#模型简化
        #print('self.processed_model:',self.processed_model)
        a = 0
        while a < len(self.processed_model):
            
            if len(self.processed_model[a][1]) > 1 :
                self.processed_model[a][0] = self.processed_model[a][0][0]#同一个x对应多个target，取拿到的第一个target作为输出
            self.processed_model[a][1] = self.processed_model[a][1][0]
            #print(f"\n\nself.processed_model:",self.processed_model)
            a = a + 1
        #print('\n\n处理后self.processed_model:',self.processed_model)
    

    def train(self,x,t,n):#n为分支最大长度
        t = np.array(t)
        self.t_type = set()
        for i in t:
            self.t_type.add(i)
        self.processed_model = self.training(x,t,n)
        
        #print(f"\n\nself.processed_model:",self.processed_model)
        self.model_process()

    def training(self,x,t,n):#n为分支最大长度(数据处理，按最大增益顺序)
        self.i = 0
        a = self.train1(x,t,n)
        #print(f"\na输出:",a)
        return a
    def train1(self,x,t,n):
        di_gui = self.one_step_highest_gain(x,t,[])
        self.i = self.i + 1
        store = []
        for h in di_gui:
            store.append(h)
        if self.i == n:
            return store
        while self.i < n:
            di_gui = []
            for h in store:
                di_gui.append(h)
            store = []
            a = 0
            for j in di_gui:
                # if len(j[0]) == 1:
                #     j[0] = j[0][0]#debug去掉多余的括号
                st = self.one_step_highest_gain(j[0],j[1],j[2])
                for c in st:
                    if len(c[0]) == 1:
                        c[0] = c[0][0]#debug去掉多余的括号
                    #print(f"c{self.i}{a}:{c}")
                    a = a+1
                    store.append(c)
            self.i = self.i + 1
            #print(f"\nstore:",store)
        #print(f"\nstore输出:",store)
        return store


                

    x_turn = []#记录x的决策顺序
    #计算并返回 最高增益x的位置（第一列是0） 按最高增益x的列排序的并分割后的x和对应的t集：
    def one_step_highest_gain(self,x,t,x_turn):#x为多列，t为单列,stop为递归终止次数
        self.x_turn = x_turn
        #print("\n\nx:",x,"\n\n\nt:",t)
        x = np.array(x)
        t = np.array(t)
        t_type = set()
        for i in t:
            t_type.add(i)
        
        #t_type = list(t_type)
        if len(t_type) == 1:# and self.cycle > 1:
            self.cycle = self.cycle - 1
            abdc = []
            for i in self.x_turn:
                abdc.append(i)
            output = [[x,t,abdc]]
            return output
    

        #最高gain相当于最小的entropy
        lowest_entropy = math.inf
        lowest_entropy_x_num = 0
        if len(x) == len(t):
            num = len(x[0])#x的列数（特征维数）
        else:
            num = 1
        a = 0
        best_classify = None
        while a < num:
            #print("x:",x[:,a],"t:",t)
            outcome = self.compute_entropy_t_when_x(np.array(x)[:,a],t)
            #print(f"entropy{a}:{outcome[0]}")
            #print(f"outcome{a}:{outcome}")
            if outcome[0] < lowest_entropy:
                lowest_entropy_x_num = a
                lowest_entropy = outcome[0]
                best_classify = outcome

            a=a+1
        #尝试做一个递归：
        #output = [lowest_entropy_x_num]
        self.x_turn.append(lowest_entropy_x_num)
        output = []
        best_classify = best_classify[1:]#去除entropy值
        #print("\n\n\nbest_classify:",best_classify)
        for i in best_classify:
            xo = []
            to = []
            for j in i:
                xo.append(x[j])
                to.append(t[j])
            
            list_x_t = [xo,to]
            #print("list_x_t:",list_x_t)
            #print("output 1:",output)
            abdc = []
            for i in self.x_turn:
                abdc.append(i)
            list_x_t.append(abdc)
            output.append(list_x_t)
            #print("output 2:",output)
        #递归：
        self.cycle = self.cycle + 1 
        #print("\noutput:",output)
        
        #print("输出：",output)
        return output
        


#    def compute_entropy_t_when_x(x,t,x_type_num,t_type_num):
 #   def compute_entropy_t_when_x(x,t,t_type):
    def compute_entropy_t_when_x(self,x,t):#x与t均为单列且长度相同
            x = x.flatten()
            x_type = set()
            t_type = set()
            num = len(t)
            for i in x:
                x_type.add(i)
            for i in t:
                t_type.add(i)
            x_type = list(x_type)
            t_type = list(t_type)
            #存储x数量各type值对应的数量和相应t的各type数量:
            data = list()
            i = 0
            while i < len(x_type):
                x_data = [0]*(1 + len(t_type))#分别存储x某type的数量及对应t各type数量
                data.append(x_data)
                i = i+1

            entropy = 0
            output = [entropy]
            for i in x_type:
                #list_ = [i]
                list_ = []
                output.append(list_)

            b = 0
            while b < num:#第b个x
                a = 0
                while a < len(x_type):#第a类x
                    if x[b] == x_type[a]:
                        data[a][0] = data[a][0] + 1
                        m = 0
                        while m < len(t_type):
                            if t[b] == t_type[m]:#第b个t为第m类t
                                data[a][m+1] = data[a][m+1] + 1 #记录对应t类别的数量
                            m = m+1
                        
                        #记录x的位置
                        output[a+1].append(b)


                    a = a + 1
                b = b+1
            #print(data)
            #计算entropy：
            a = 0
            
            #print(data)
            while a < len(x_type):
                entropy_t_when_x_a = 0
                b = 1
                #print('死循环1')
                while b <= len(t_type):
                    if data[a][b]/data[a][0] != 0:    
                        entropy_t_when_x_a = entropy_t_when_x_a - (data[a][b]/data[a][0])*math.log2(data[a][b]/data[a][0])
                    b = b+1
                entr = (data[a][0] / num)*entropy_t_when_x_a

                #print("first: ", entr)


                entropy = entropy + entr

                #print("first: ", entropy)

                output[0] = entropy
                a=a+1
            #print("output:",output)
            return output#output为列表，第一个量为entropy，后面若干个量为list（""list的第一位存x的type"(删掉了），第二位及以后"(删掉了）存x某个type的所有位置）
        


lenses = np.loadtxt(filepath,dtype = int,usecols=(1,2,3,4,5))
#lenses = lenses[3:20,:]
#print(lenses)
x = lenses[:,:4]
target = lenses[:,4:].flatten()

model = DecisionTree()
d = 0
x_random = []
while d < 100:
    b = 0
    a = []
    while b < 4:
        b = b+1
        a.append(random.randrange(1,4))#随机生成一个样本
    x_random.append(a)
    d = d+1

model.train(x,target,len(x[0]))
#predict方法只能预测样本数量大于等于2的情况，样本为1时用predict_one_value方法预测
print(f"\n训练后的决策树预测模型：{model.processed_model}\n数列里的每个元素记录了分支，每个元素的第三项记录了决策顺序，第一项记录了在该决策顺序下对应的x值，第二项记录了x值对应的预测值")
print("\n输入为原样本特征时的输出结果",model.predict(x))
y = model.predict(x_random)
print(f"\n随机样本:{x_random}\n预测值：{y}")
#number = lenses.length



训练后的决策树预测模型：[[array([1, 1, 1, 1]), 3, [3]], [array([1, 1, 1, 2]), 2, [3, 2, 0]], [array([2, 1, 1, 2]), 2, [3, 2, 0]], [array([3, 1, 1, 2]), 3, [3, 2, 0, 1]], [array([3, 2, 1, 2]), 2, [3, 2, 0, 1]], [array([1, 1, 2, 2]), 1, [3, 2, 1]], [array([1, 2, 2, 2]), 1, [3, 2, 1, 0]], [array([2, 2, 2, 2]), 3, [3, 2, 1, 0]], [array([3, 2, 2, 2]), 3, [3, 2, 1, 0]]]
数列里的每个元素记录了分支，每个元素的第三项记录了决策顺序，第一项记录了在该决策顺序下对应的x值，第二项记录了x值对应的预测值

输入为原样本特征时的输出结果 [3, 2, 3, 1, 3, 2, 3, 1, 3, 2, 3, 1, 3, 2, 3, 3, 3, 3, 3, 1, 3, 2, 3, 3]

随机样本:[[1, 2, 2, 2], [2, 2, 1, 3], [1, 2, 3, 3], [1, 1, 2, 3], [2, 2, 1, 1], [3, 1, 3, 1], [2, 2, 1, 2], [2, 3, 3, 2], [2, 1, 1, 3], [2, 3, 1, 3], [2, 2, 2, 2], [1, 3, 3, 1], [3, 1, 3, 1], [1, 2, 3, 1], [3, 1, 3, 1], [3, 3, 1, 3], [3, 1, 3, 1], [2, 1, 3, 1], [3, 1, 3, 3], [3, 3, 3, 1], [2, 3, 2, 1], [2, 3, 3, 2], [3, 1, 3, 2], [1, 1, 2, 2], [3, 1, 1, 3], [3, 3, 3, 3], [3, 3, 3, 1], [2, 2, 3, 3], [2, 2, 2, 3], [1, 1, 3, 3], [1, 3, 3, 2], [3, 2, 3, 2], [3, 1, 2, 2], [1, 1, 2, 1], [2, 2, 2